In [2]:
import pandas as pd
import numpy as np

In [3]:
results_df = pd.read_csv("Rahul RAG llm evaluationmetric results_df.csv")

In [4]:
results_df

,id,dataset_name,question,original_documents,original_response,retrieved_documents,qwen_05B,DeepSeek,llm_response,original_documents_sentences,...,retrieved_relevant_documents_sentences_identifiers,context_relevance,utilization_score,completeness_score,is_adherent,noisy_response,noise_robustness,negative_rejection_score,counterfactual_robustness_score,info_integration_score
0,358,covidqa,What role does T-cell count play in severe hum...,['Title: Emergent severe acute respiratory dis...,The T-cell count plays a crucial role in sever...,['Title: Emergent severe acute respiratory dis...,What role does T-cell count play in severe hum...,Question: What role does T-cell count play in ...,"The semicontaminated area, as mentioned in the...","[[['0a', 'Title: Emergent severe acute respira...",...,"[""'0a'"", ""'0b'"", ""'0c'"", ""'0d'"", ""'0e'"", ""'0f'...",0.526316,1.445958,1.000000,True,"[[['0a', 'Title: Emergent severe acute respira...",0.0,50.0,0.75,NaN
1,457,covidqa,What is MVO?,['Title: Gene Knockdowns in Adult Animals: PPM...,MVO is not defined or mentioned in the provide...,['Intraocular pressure (IOP) is the fluid pres...,What is MVO?['Title: Gene Knockdowns in Adult ...,Question: What is MVO?\nContext: ['Title: Gene...,"Based on the context provided, as of 21 Februa...","[[['0a', 'Title: Gene Knockdowns in Adult Anim...",...,[],0.000000,1.445958,0.000000,True,[],0.0,50.0,0.75,NaN
2,1172,covidqa,What have sero-surveys of MERS virus found?,"['Title: MERS coronavirus: diagnostics, epidem...",Sero-surveys of MERS virus found no sign of ME...,"['Title: MERS coronavirus: diagnostics, epidem...",What have sero-surveys of MERS virus found?['T...,Question: What have sero-surveys of MERS virus...,"Based on the context provided, sero-surveys of...","[[['0a', 'Title: MERS coronavirus: diagnostics...",...,"[""'0a'"", ""'0b'"", ""'0c'"", ""'0d'"", ""'0e'"", ""'0f'""]",0.260870,1.445958,0.000000,True,"[[['0a', 'Title: MERS coronavirus: diagnostics...",0.0,50.0,0.75,NaN
3,1033,covidqa,Where was alpha-tubulin found least abundantly...,['Title: Nucleolar Protein Trafficking in Resp...,Alpha-tubulin was found least abundantly in th...,['Title: Nucleolar Protein Trafficking in Resp...,Where was alpha-tubulin found least abundantly...,Question: Where was alpha-tubulin found least ...,Alpha-tubulin was found to be weakly detected ...,"[[['0a', 'Title: Nucleolar Protein Trafficking...",...,"[""'0a'"", ""'0b'"", ""'0c'"", ""'0d'"", ""'0e'"", ""'0f'...",0.611111,1.445958,0.250000,False,"[[['0a', 'Title: Nucleolar Protein Trafficking...",0.0,50.0,0.75,NaN
4,1273,covidqa,What does the semicontaminated area include?,['Title: The Battle Against Coronavirus Diseas...,The semicontaminated area includes the fever-C...,['Title: The Battle Against Coronavirus Diseas...,What does the semicontaminated area include?['...,Question: What does the semicontaminated area ...,"In the context provided, T-cell count seems to...","[[['0a', 'Title: The Battle Against Coronaviru...",...,"[""'0a'"", ""'0b'""]",0.200000,1.445958,0.250000,True,"[[['0a', 'Title: The Battle Against Coronaviru...",0.0,50.0,0.75,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,finqa_test_96,tatqa,When are multi-year agreement customers invoiced?,"['Contract Balances [[""(In millions)"", """", """",...",Multi-year agreement customers are generally i...,['Timing of revenue recognition may differ fro...,When are multi-year agreement customers invoic...,Question: When are multi-year agreement custom...,Multi-year agreement customers are typically i...,"[[['0a', 'Contract Balances [[""(In millions)"",...",...,"[""'0a'""]",0.200000,1.445958,0.000000,True,"[[['1a', 'Timing of revenue recognition may di...",0.0,50.0,0.75,NaN
104,finqa_test_199,tatqa,What is the percentage change in the value of ...,"['Inventories and Inventory Valuation [[""March...",To calculate the percentage change in the valu...,"['At December 31, 2019 and 2018, the Company h...",What i

In [5]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 24 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   id                                                  108 non-null    object 
 1   dataset_name                                        108 non-null    object 
 2   question                                            108 non-null    object 
 3   original_documents                                  108 non-null    object 
 4   original_response                                   108 non-null    object 
 5   retrieved_documents                                 108 non-null    object 
 6   qwen_05B                                            108 non-null    object 
 7   DeepSeek                                            108 non-null    object 
 8   llm_response                                        107 non-null    object 
 9  

Rahul checkpoint 5

In [6]:
#Deployment
!pip install gradio
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [7]:
# Define the list of available LLMs
available_llms = ["QWEN", "DeepSeek", "Mistral"]

In [10]:
# Function to get the LLM response and evaluation metrics
def get_llm_response_and_metrics(question, llm_name):
    # Filter the DataFrame based on the selected question and LLM
    filtered_df = results_df[results_df['question'] == question]
    llm_response = filtered_df['llm_response'].iloc[0]

    # Extract the evaluation metrics from the filtered DataFrame
    evaluation_metrics = filtered_df[[
        'context_relevance', 'utilization_score', 'completeness_score',
        'is_adherent'
    ]]

    # Convert the evaluation metrics to a DataFrame for display
    evaluation_df = pd.DataFrame(evaluation_metrics)
    evaluation_df = evaluation_df.transpose().reset_index()
    evaluation_df.columns = ['Metric', 'Value']

    return llm_response, evaluation_df

In [11]:
# Get the list of unique questions from the DataFrame
available_questions = results_df['question'].unique().tolist()

# Create a Gradio interface
iface = gr.Interface(
    fn=get_llm_response_and_metrics,
    inputs=[
        gr.Dropdown(available_questions, label="Select a Question"),
        gr.Radio(available_llms, label="Select LLM")
    ],
    outputs=[
        gr.Textbox(label="LLM Response"),
        gr.DataFrame(label="Evaluation Metrics")
    ],
    title="RAG Evaluation Metrics",
    description="Select a question and LLM to see the response and evaluation metrics in a table format."
)

# Launch the interface
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://052b1c09047a1a1b9f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


https://github.com/RahulMAIpy/RAG_Capstone_grp2